In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
import tensorflow as tf
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if not tf.__version__ == '2.15.0':
    printmd('<<<<<!!!!! ERROR !!!! please upgrade to TensorFlow 2.17.0, or restart your Kernel (Kernel->Restart & Clear Output)>>>>>')


# <span style="color:red"><<<<<!!!!! ERROR !!!! please upgrade to TensorFlow 2.17.0, or restart your Kernel (Kernel->Restart & Clear Output)>>>>></span>

In [ ]:
width = 28 # width of the image in pixels
height = 28 # height of the image in pixels
flat = width * height # number of pixels in one image
class_output = 10 # number of possible classifications for the problem

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape, y_test.shape

((60000, 28, 28), (10000,))

In [ ]:
x_train[0]/255

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
print(x_train.shape)

(60000, 28, 28)


In [ ]:
print(y_train.shape)

(60000,)


In [ ]:
y_train[:50]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0,
       9, 1, 1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9,
       3, 9, 8, 5, 9, 3], dtype=uint8)

In [ ]:
print("categorical labels")
print(y_train[0:5])

# make labels one hot encoded
# Convert y_train and y_test to integers before applying one_hot encoding
y_train = tf.one_hot(tf.cast(y_train, dtype=tf.int32), 10) # Convert to int32
y_test = tf.one_hot(tf.cast(y_test, dtype=tf.int32), 10) # Convert to int32

print("one hot encoded labels")
print(y_train[0:5])

categorical labels
[5 0 4 1 9]
one hot encoded labels
tf.Tensor(
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]], shape=(5, 10), dtype=float32)


In [ ]:
print("number of training examples:" , x_train.shape[0])
print("number of test examples:" , x_test.shape[0])

number of training examples: 60000
number of test examples: 10000


In [ ]:
x_image_train = tf.reshape(x_train, [-1,28,28,1])
x_image_train = tf.cast(x_image_train, 'float32')

x_image_test = tf.reshape(x_test, [-1,28,28,1])
x_image_test = tf.cast(x_image_test, 'float32')

#creating new dataset with reshaped inputs
train_ds2 = tf.data.Dataset.from_tensor_slices((x_image_train, y_train)).batch(50)
test_ds2 = tf.data.Dataset.from_tensor_slices((x_image_test, y_test)).batch(50)


In [ ]:
W_conv1 = tf.Variable(tf.random.truncated_normal([5, 5, 1, 32], stddev=0.1, seed=0))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) # need 32 biases for 32 outputs


In [ ]:
def convolve1(x):
    return(
        tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)


In [ ]:
def h_conv1(x): return(tf.nn.relu(convolve1(x)))
def conv1(x):
    return tf.nn.max_pool(h_conv1(x), ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


In [ ]:
W_conv2 = tf.Variable(tf.random.truncated_normal([5, 5, 32, 64], stddev=0.1, seed=1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64])) #need 64 biases for 64 outputs


In [ ]:
def convolve2(x):
    return(
    tf.nn.conv2d(conv1(x), W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)


In [ ]:
def h_conv2(x):  return tf.nn.relu(convolve2(x))


In [ ]:
def conv2(x):
    return(
    tf.nn.max_pool(h_conv2(x), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME'))
def layer2_matrix(x): return tf.reshape(conv2(x), [-1, 7 * 7 * 64])


In [ ]:
W_fc1 = tf.Variable(tf.random.truncated_normal([7 * 7 * 64, 1024], stddev=0.1, seed = 2))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024])) # need 1024 biases for 1024 outputs

In [ ]:
def fcl(x): return tf.matmul(layer2_matrix(x), W_fc1) + b_fc1

In [ ]:
def h_fc1(x): return tf.nn.relu(fcl(x))

In [ ]:
def layer_drop(x): return tf.nn.dropout(h_fc1(x), keep_prob)

In [ ]:
W_fc2 = tf.Variable(tf.random.truncated_normal([1024, 10], stddev=0.1, seed = 2)) #1024 neurons
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10])) # 10 possibilities for digits [0,1,2,3,4,5,6,7,8,9]

In [ ]:
def fc(x): return tf.matmul(layer_drop(x), W_fc2) + b_fc2

In [ ]:
def y_CNN(x): return tf.nn.softmax(fc(x))

In [ ]:
import numpy as np
layer4_test =[[0.9, 0.1, 0.1],[0.9, 0.1, 0.1]]
y_test=[[1.0, 0.0, 0.0],[1.0, 0.0, 0.0]]
np.mean( -np.sum(y_test * np.log(layer4_test),1))

0.10536051565782628

In [ ]:
def cross_entropy(y_label, y_pred):
    return (-tf.reduce_sum(y_label * tf.math.log(y_pred + 1.e-10)))

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
variables = [W_conv1, b_conv1, W_conv2, b_conv2,
             W_fc1, b_fc1, W_fc2, b_fc2, ]

def train_step(x, y):
    with tf.GradientTape() as tape:
        current_loss = cross_entropy( y, y_CNN( x ))
        grads = tape.gradient( current_loss , variables )
        optimizer.apply_gradients( zip( grads , variables ) )
        return current_loss.numpy()

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_CNN(x_image_train), axis=1), tf.argmax(y_train, axis=1))

In [ ]:
print("number of training examples:" , x_train.shape[0])
print("number of test examples:" , x_test.shape[0])
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(50)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(50)
train_ds

# showing an example of the Flatten class and operation
from tensorflow.keras.layers import Flatten
flatten = Flatten(dtype='float32')

"original data shape"
#print(x_train[0])

"flattened shape"
print(flatten(x_train)[0])

In [ ]:
# Weight tensor
W = tf.Variable(tf.zeros([784, 10], tf.float32))
# Bias tensor
b = tf.Variable(tf.zeros([10], tf.float32))

In [ ]:
def forward(x):
    return tf.matmul(x,W) + b

In [ ]:
# a sample softmax calculation on an input vector
vector = [10, 0.2, 8]
softmax = tf.nn.softmax(vector)
print("softmax calculation")
print(softmax.numpy())
print("verifying normalization")
print(tf.reduce_sum(softmax))
print("finding vector with largest value (label assignment)")
print("category", tf.argmax(softmax).numpy())

In [ ]:
def activate(x):
    return tf.nn.softmax(forward(x))

In [ ]:
def model(x):
    x = flatten(x)
    return activate(x)

In [ ]:
def cross_entropy(y_label, y_pred):
    return (-tf.reduce_sum(y_label * tf.math.log(y_pred + 1.e-10)))
# addition of 1e-10 to prevent errors in zero calculations

# current loss function for unoptimized model
cross_entropy(y_train, model(x_train)).numpy()

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.25)

In [ ]:
def train_step(x, y ):
    with tf.GradientTape() as tape:
        #compute loss function
        current_loss = cross_entropy( y, model(x))
        # compute gradient of loss
        #(This is automatic! Even with specialized funcctions!)
        grads = tape.gradient( current_loss , [W,b] )
        # Apply SGD step to our Variables W and b
        optimizer.apply_gradients( zip( grads , [W,b] ) )
    return current_loss.numpy()

In [ ]:
# zeroing out weights in case you want to run this cell multiple times
# Weight tensor
W = tf.Variable(tf.zeros([784, 10],tf.float32))
# Bias tensor
b = tf.Variable(tf.zeros([10],tf.float32))

loss_values=[]
accuracies = []
epochs = 100

for i in range(epochs):
    j=0
    # each batch has 50 examples
    for x_train_batch, y_train_batch in train_ds:
        j+=1
        current_loss = train_step(x_train_batch, y_train_batch)
        if j%500==0: #reporting intermittent batch statistics
            print("epoch ", str(i), "batch", str(j), "loss:", str(current_loss) )

    # collecting statistics at each epoch...loss function and accuracy
    #  loss function
    current_loss = cross_entropy( y_train, model( x_train )).numpy()
    loss_values.append(current_loss)
    correct_prediction = tf.equal(tf.argmax(model(x_train), axis=1),
                                  tf.argmax(y_train, axis=1))
    #  accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)).numpy()
    accuracies.append(accuracy)
    print("end of epoch ", str(i), "loss", str(current_loss), "accuracy", str(accuracy) )
correct_prediction_train = tf.equal(tf.argmax(model(x_train), axis=1),tf.argmax(y_train,axis=1))
accuracy_train = tf.reduce_mean(tf.cast(correct_prediction_train, tf.float32)).numpy()

correct_prediction_test = tf.equal(tf.argmax(model(x_test), axis=1),tf.argmax(y_test, axis=1))
accuracy_test = tf.reduce_mean(tf.cast(correct_prediction_test, tf.float32)).numpy()

print("training accuracy", accuracy_train)
print("test accuracy", accuracy_test)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
#print(loss_values)
plt.plot(loss_values,'-ro')
plt.title("loss per epoch")
plt.xlabel("epoch")
plt.ylabel("loss")

In [ ]:
plt.plot(accuracies,'-ro')
plt.title("accuracy per epoch")
plt.xlabel("epoch")
plt.ylabel("accuracy")

In [ ]:
width = 28 # width of the image in pixels
height = 28 # height of the image in pixels
flat = width * height # number of pixels in one image
class_output = 10 # number of possible classifications for the problem

In [ ]:
x_image_train = tf.reshape(x_train, [-1,28,28,1])
x_image_train = tf.cast(x_image_train, 'float32')

x_image_test = tf.reshape(x_test, [-1,28,28,1])
x_image_test = tf.cast(x_image_test, 'float32')

#creating new dataset with reshaped inputs
train_ds2 = tf.data.Dataset.from_tensor_slices((x_image_train, y_train)).batch(50)
test_ds2 = tf.data.Dataset.from_tensor_slices((x_image_test, y_test)).batch(50)

In [ ]:
x_image_train = tf.slice(x_image_train,[0,0,0,0],[10000, 28, 28, 1])
y_train = tf.slice(y_train,[0,0],[10000, 10])

NameError: name 'tf' is not defined

In [ ]:
W_conv1 = tf.Variable(tf.random.truncated_normal([5, 5, 1, 32], stddev=0.1, seed=0))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) # need 32 biases for 32 outputs

NameError: name 'tf' is not defined

In [ ]:

W_conv1 = tf.Variable(tf.random.truncated_normal([5, 5, 1, 32], stddev=0.1, seed=0))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) # need 32 biases for 32 outputs

Text cell <N7MSGrZWgXWq>
# %% [markdown]
<img src="https://ibm.box.com/shared/static/vn26neef1nnv2oxn5cb3uueowcawhkgb.png" style="width: 800px; height: 400px;" alt="HTML5 Icon" >

</h4>Convolve with weight tensor and add biases.</h4>

To create convolutional layer, we use <b>tf.nn.conv2d</b>. It computes a 2-D convolution given 4-D input and filter tensors.

Inputs:

-   tensor of shape [batch, in_height, in_width, in_channels]. x of shape [batch_size,28 ,28, 1]
-   a filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]. W is of size [5, 5, 1, 32]
-   stride which is  [1, 1, 1, 1]. The convolutional layer, slides the "kernel window" across the input tensor. As the input tensor has 4 dimensions:  [batch, height, width, channels], then the convolution operates on a 2D window on the height and width dimensions. **strides** determines how much the window shifts by in each of the dimensions. As the first and last dimensions are related to batch and channels, we set the stride to 1. But for second and third dimension, we could set other values, e.g. [1, 2, 2, 1]

Process:

-   Change the filter to a 2-D matrix with shape [5\*5\*1,32]
-   Extracts image patches from the input tensor to form a _virtual_ tensor of shape `[batch, 28, 28, 5*5*1]`.
-   For each batch, right-multiplies the filter matrix and the image vector.

Output:

-   A `Tensor` (a 2-D convolution) of size tf.Tensor 'add_7:0' shape=(?, 28, 28, 32)- Notice: the output of the first convolution layer is 32 [28x28] images. Here 32 is considered as volume/depth of the output image.


Code cell <gqFEpMb9gXWq>
# %% [code]
def convolve1(x):
    return(
        tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

Text cell <1O_PkrmXgXWq>
# %% [markdown]
<img src="https://ibm.box.com/shared/static/iizf4ui4b2hh9wn86pplqxu27ykpqci9.png" style="width: 800px; height: 400px;" alt="HTML5 Icon" >


Text cell <li8fq8rngXWq>
# %% [markdown]
<h4>Apply the ReLU activation Function</h4>


Text cell <VYGMET7dgXWq>
# %% [markdown]
In this step, we just go through all outputs convolution layer, <b>convolve1</b>, and wherever a negative number occurs, we swap it out for a 0. It is called ReLU activation Function.<br> Let f(x) is a ReLU activation function $f(x) = max(0,x)$.


Code cell <6m6pW6HDgXWq>
# %% [code]
def h_conv1(x): return(tf.nn.relu(convolve1(x)))

Text cell <xKbplh40gXWr>
# %% [markdown]
<h4>Apply the max pooling</h4>


Text cell <SnwWB-yfgXWr>
# %% [markdown]
<b>max pooling</b> is a form of non-linear down-sampling. It partitions the input image into a set of rectangles and, and then find the maximum value for that region.

Lets use <b>tf.nn.max_pool</b> function to perform max pooling.
<b>Kernel size:</b> 2x2 (if the window is a 2x2 matrix, it would result in one output pixel)
<b>Strides:</b> dictates the sliding behaviour of the kernel. In this case it will move 2 pixels everytime, thus not overlapping. The input is a matrix of size 28x28x32, and the output would be a matrix of size 14x14x32.

<img src="https://ibm.box.com/shared/static/kmaja90mn3aud9mro9cn8pbbg1h5pejy.png" alt="HTML5 Icon" style="width: 800px; height: 400px;">


Code cell <S32MqUsmgXWr>
# %% [code]
def conv1(x):
    return tf.nn.max_pool(h_conv1(x), ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

Text cell <WNwOLTYTgXWr>
# %% [markdown]
First layer completed


Text cell <r_K3CB5XgXWr>
# %% [markdown]
<h3>Convolutional Layer 2</h3>
<h4>Weights and Biases of kernels</h4>


Text cell <pFpT9BzfgXWr>
# %% [markdown]
We apply the convolution again in this layer. Lets look at the second layer kernel:

-   Filter/kernel: 5x5 (25 pixels)
-   Input channels: 32 (from the 1st Conv layer, we had 32 feature maps)
-   64 output feature maps

<b>Notice:</b> here, the input image is [14x14x32], the filter is [5x5x32], we use 64 filters of size [5x5x32], and the output of the convolutional layer would be 64 convolved image, [14x14x64].

<b>Notice:</b> the convolution result of applying a filter of size [5x5x32] on image of size [14x14x32] is an image of size [14x14x1], that is, the convolution is functioning on volume.


Code cell <OaIfgHiggXWr>
# %% [code]
W_conv2 = tf.Variable(tf.random.truncated_normal([5, 5, 32, 64], stddev=0.1, seed=1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64])) #need 64 biases for 64 outputs

Text cell <gxWC0p-ggXWr>
# %% [markdown]
<h4>Convolve image with weight tensor and add biases.</h4>


Code cell <ouTXIWazgXWr>
# %% [code]
def convolve2(x):
    return(
    tf.nn.conv2d(conv1(x), W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

Text cell <CWvK6KCkgXWr>
# %% [markdown]
<h4>Apply the ReLU activation Function</h4>


Code cell <-aw3d0L_gXWs>
# %% [code]
def h_conv2(x):  return tf.nn.relu(convolve2(x))

Text cell <uCLzaMdSgXWs>
# %% [markdown]
<h4>Apply the max pooling</h4>


Code cell <v-rG95IfgXWs>
# %% [code]
def conv2(x):
    return(
    tf.nn.max_pool(h_conv2(x), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME'))

Text cell <bSbFvUBGgXWs>
# %% [markdown]
Second layer completed. So, what is the output of the second layer, layer2?

-   it is 64 matrix of [7x7]


Text cell <TIFqUgSngXWs>
# %% [markdown]
<h3>Fully Connected Layer</h3>


Text cell <_MoA39j2gXWs>
# %% [markdown]
You need a fully connected layer to use the Softmax and create the probabilities in the end. Fully connected layers take the high-level filtered images from previous layer, that is all 64 matrices, and convert them to a flat array.

So, each matrix [7x7] will be converted to a matrix of [49x1], and then all of the 64 matrix will be connected, which make an array of size [3136x1]. We will connect it into another layer of size [1024x1]. So, the weight between these 2 layers will be [3136x1024]

<img src="https://ibm.box.com/shared/static/pr9mnirmlrzm2bitf1d4jj389hyvv7ey.png" alt="HTML5 Icon" style="width: 800px; height: 400px;">


Text cell <sI1-fURKgXWs>
# %% [markdown]
<h4>Flattening Second Layer</h4>


Code cell <MOqZ9lqzgXWs>
# %% [code]
def layer2_matrix(x): return tf.reshape(conv2(x), [-1, 7 * 7 * 64])

Text cell <ULBtCUiCgXWs>
# %% [markdown]
<h4>Weights and Biases between layer 2 and 3</h4>


Text cell <u_6xQkjogXWt>
# %% [markdown]
Composition of the feature map from the last layer (7x7) multiplied by the number of feature maps (64); 1027 outputs to Softmax layer


Code cell <YSo29MN8gXWt>
# %% [code]
W_fc1 = tf.Variable(tf.random.truncated_normal([7 * 7 * 64, 1024], stddev=0.1, seed = 2))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024])) # need 1024 biases for 1024 outputs

Text cell <w8ixW7dcgXWt>
# %% [markdown]
<h4>Matrix Multiplication (applying weights and biases)</h4>


Code cell <7n7A0OAEgXWt>
# %% [code]
def fcl(x): return tf.matmul(layer2_matrix(x), W_fc1) + b_fc1

Text cell <E8denUdrgXWt>
# %% [markdown]
<h4>Apply the ReLU activation Function</h4>


Code cell <R9xr6HmpgXWt>
# %% [code]
def h_fc1(x): return tf.nn.relu(fcl(x))

Text cell <2-KodCLRgXWt>
# %% [markdown]
Third layer completed


Text cell <ZZTtPm3rgXWu>
# %% [markdown]
<h4>Dropout Layer, Optional phase for reducing overfitting</h4>


Text cell <5fiZxhTpgXWu>
# %% [markdown]
It is a phase where the network "forget" some features. At each training step in a mini-batch, some units get switched off randomly so that it will not interact with the network. That is, it weights cannot be updated, nor affect the learning of the other network nodes.  This can be very useful for very large neural networks to prevent overfitting.


Code cell <94sxZAQxgXWu>
# %% [code]
keep_prob=0.5
def layer_drop(x): return tf.nn.dropout(h_fc1(x), keep_prob)

Text cell <nOOmzL9HgXWu>
# %% [markdown]
<h4>Readout Layer (Softmax Layer)</h4>


Text cell <XfdJfz2ogXWu>
# %% [markdown]
Type: Softmax, Fully Connected Layer.


Text cell <fsHFatuEgXWv>
# %% [markdown]
<h4>Weights and Biases</h4>


Text cell <Hn_IJjecgXWv>
# %% [markdown]
In last layer, CNN takes the high-level filtered images and translate them into votes using softmax.
Input channels: 1024 (neurons from the 3rd Layer); 10 output features


Code cell <TwXVFcG7gXWv>
# %% [code]
W_fc2 = tf.Variable(tf.random.truncated_normal([1024, 10], stddev=0.1, seed = 2)) #1024 neurons
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10])) # 10 possibilities for digits [0,1,2,3,4,5,6,7,8,9]

Text cell <hLV2jgrigXWv>
# %% [markdown]
<h4>Matrix Multiplication (applying weights and biases)</h4>


Code cell <UYqsUpF-gXWv>
# %% [code]
def fc(x): return tf.matmul(layer_drop(x), W_fc2) + b_fc2

Text cell <qGfira_WgXWv>
# %% [markdown]
<h4>Apply the Softmax activation Function</h4>
<b>softmax</b> allows us to interpret the outputs of <b>fcl4</b> as probabilities. So, <b>y_conv</b> is a tensor of probabilities.


Code cell <KSKsNSzSgXWv>
# %% [code]
def y_CNN(x): return tf.nn.softmax(fc(x))

Text cell <iv7chb4ggXWv>
# %% [markdown]
* * *


Text cell <aA1ZqJpigXWv>
# %% [markdown]
<a id="ref7"></a>

<h2>Summary of the Deep Convolutional Neural Network</h2>


Text cell <jbfw1ps_gXWw>
# %% [markdown]
Now is time to remember the structure of  our network


Text cell <h6AnNWsIgXWw>
# %% [markdown]
#### 0) Input - MNIST dataset

#### 1) Convolutional and Max-Pooling

#### 2) Convolutional and Max-Pooling

#### 3) Fully Connected Layer

#### 4) Processing - Dropout

#### 5) Readout layer - Fully Connected

#### 6) Outputs - Classified digits


Text cell <ssjPiIXMgXWw>
# %% [markdown]
* * *


Text cell <WemsVCR6gXWw>
# %% [markdown]
<a id="ref8"></a>

<h2>Define functions and train the model</h2>


Text cell <plztfol9gXWw>
# %% [markdown]
<h4>Define the loss function</h4>

We need to compare our output, layer4 tensor, with ground truth for all mini_batch. we can use <b>cross entropy>/b> to see how bad our CNN is working - to measure the error at a softmax layer.

The following code shows an toy sample of cross-entropy for a mini-batch of size 2 which its items have been classified. You can run it (first change the cell type to <b>code</b> in the toolbar) to see how cross entropy changes.


Text cell <vTngITP5gXWw>
# %% [raw]
import numpy as np
layer4_test =[[0.9, 0.1, 0.1],[0.9, 0.1, 0.1]]
y_test=[[1.0, 0.0, 0.0],[1.0, 0.0, 0.0]]
np.mean( -np.sum(y_test * np.log(layer4_test),1))

Text cell <muWAhNOFgXWw>
# %% [markdown]
<b>reduce_sum</b> computes the sum of elements of <b>(y_ * tf.log(layer4)</b> across second dimension of the tensor, and <b>reduce_mean</b> computes the mean of all elements in the tensor..

$$ CrossEntropy = \sum{y_{Label}\cdot \log(y_{Prediction})}$$


Code cell <3TNInHf8gXWw>
# %% [code]
def cross_entropy(y_label, y_pred):
    return (-tf.reduce_sum(y_label * tf.math.log(y_pred + 1.e-10)))

Text cell <y9VfJgNRgXWw>
# %% [markdown]
<h4>Define the optimizer</h4>

It is obvious that we want minimize the error of our network which is calculated by cross_entropy metric. To solve the problem, we have to compute gradients for the loss (which is minimizing the cross-entropy) and apply gradients to variables. It will be done by an optimizer: GradientDescent or Adagrad.


Code cell <T6GICWzbgXWw>
# %% [code]
optimizer = tf.keras.optimizers.Adam(1e-4)

Text cell <N2FtrishgXWw>
# %% [markdown]
Following the convention of our first example, we will use `GradientTape` to define a model.


Code cell <_LpdCGQ6gXWw>
# %% [code]
variables = [W_conv1, b_conv1, W_conv2, b_conv2,
             W_fc1, b_fc1, W_fc2, b_fc2, ]

def train_step(x, y):
    with tf.GradientTape() as tape:
        current_loss = cross_entropy( y, y_CNN( x ))
        grads = tape.gradient( current_loss , variables )
        optimizer.apply_gradients( zip( grads , variables ) )
        return current_loss.numpy()


Code cell <mGOX6_1xgXWx>
# %% [code]
"""results = []
increment = 1000
for start in range(0,60000,increment):
    s = tf.slice(x_image_train,[start,0,0,0],[start+increment-1, 28, 28, 1])
    t = y_CNN(s)
    #results.append(t)
"""
Execution output
0KB
	text/plain
		'results = []\nincrement = 1000\nfor start in range(0,60000,increment):\n    s = tf.slice(x_image_train,[start,0,0,0],[start+increment-1, 28, 28, 1])\n    t = y_CNN(s)\n    #results.append(t)\n'

Text cell <0XmkgzTogXWx>
# %% [markdown]
<h4>Define prediction</h4>
Do you want to know how many of the cases in a mini-batch has been classified correctly? lets count them.


Code cell <7JK6wYWBgXWx>
# %% [code]
correct_prediction = tf.equal(tf.argmax(y_CNN(x_image_train), axis=1), tf.argmax(y_train, axis=1))

Text cell <v1u_LpFtgXWx>
# %% [markdown]
<h4>Define accuracy</h4>
It makes more sense to report accuracy using average of correct cases.


Code cell <C9R0d4V3gXWx>
# %% [code]
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float32'))

Text cell <fW0-o9TugXWx>
# %% [markdown]
<h4>Run session, train</h4>


Text cell <o0qG86hUgXWx>
# %% [markdown]
<i>If you want a fast result (<b>it might take sometime to train it</b>)</i>


Code cell <r0mE9RrfgXWx>
# %% [code]
loss_values=[]
accuracies = []
epochs = 1

for i in range(epochs):
    j=0
    # each batch has 50 examples
    for x_train_batch, y_train_batch in train_ds2:
        j+=1
        current_loss = train_step(x_train_batch, y_train_batch)
        if j%50==0: #reporting intermittent batch statistics
            correct_prediction = tf.equal(tf.argmax(y_CNN(x_train_batch), axis=1),
                                  tf.argmax(y_train_batch, axis=1))
            #  accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)).numpy()
            print("epoch ", str(i), "batch", str(j), "loss:", str(current_loss),
                     "accuracy", str(accuracy))

    current_loss = cross_entropy( y_train, y_CNN( x_image_train )).numpy()
    loss_values.append(current_loss)
    correct_prediction = tf.equal(tf.argmax(y_CNN(x_image_train), axis=1),
                                  tf.argmax(y_train, axis=1))
    #  accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)).numpy()
    accuracies.append(accuracy)
    print("end of epoch ", str(i), "loss", str(current_loss), "accuracy", str(accuracy) )
Execution output
1KB
	Stream
		epoch  0 batch 50 loss: 133.42038 accuracy 0.46
		epoch  0 batch 100 loss: 61.91632 accuracy 0.72
		epoch  0 batch 150 loss: 47.366875 accuracy 0.84
		epoch  0 batch 200 loss: 20.221106 accuracy 0.88
		epoch  0 batch 250 loss: 32.49733 accuracy 0.82
		epoch  0 batch 300 loss: 18.42619 accuracy 0.88
		epoch  0 batch 350 loss: 28.903078 accuracy 0.82
		epoch  0 batch 400 loss: 9.930068 accuracy 0.9
		epoch  0 batch 450 loss: 31.451693 accuracy 0.84
		epoch  0 batch 500 loss: 18.593328 accuracy 0.92
		epoch  0 batch 550 loss: 15.98876 accuracy 0.9
		epoch  0 batch 600 loss: 21.600845 accuracy 0.9
		epoch  0 batch 650 loss: 19.24796 accuracy 0.86
		epoch  0 batch 700 loss: 9.7369175 accuracy 0.92
		epoch  0 batch 750 loss: 33.084084 accuracy 0.92
		epoch  0 batch 800 loss: 11.409676 accuracy 0.88
		epoch  0 batch 850 loss: 14.52256 accuracy 0.9
		epoch  0 batch 900 loss: 10.0998955 accuracy 0.9
		epoch  0 batch 950 loss: 15.64908 accuracy 0.9
		epoch  0 batch 1000 loss: 12.104089 accuracy 0.88
		epoch  0 batch 1050 loss: 5.700769 accuracy 0.98
		epoch  0 batch 1100 loss: 13.968972 accuracy 0.9
		epoch  0 batch 1150 loss: 8.938397 accuracy 0.94
		epoch  0 batch 1200 loss: 2.09421 accuracy 1.0
		end of epoch  0 loss 12035.083 accuracy 0.93768334

Text cell <gt3anZNcgXWy>
# %% [markdown]
<i>PS. If you have problems running this notebook, please shutdown all your Jupyter runnning notebooks, clear all cells outputs and run each cell only after the completion of the previous cell.</i>


Text cell <AC07WlGCgXWy>
# %% [markdown]
<hr>


Text cell <dry8DrVRgXWy>
# %% [markdown]
<a id="ref9"></a>

<h2>Evaluate the model</h2>


Text cell <zrx4fgWfgXWy>
# %% [markdown]
Print the evaluation to the user


Code cell <m0V8WGMFgXWy>
# %% [code]
j=0
acccuracies=[]
# evaluate accuracy by batch and average...reporting every 100th batch
for x_train_batch, y_train_batch in train_ds2:
        j+=1
        correct_prediction = tf.equal(tf.argmax(y_CNN(x_train_batch), axis=1),
                                  tf.argmax(y_train_batch, axis=1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)).numpy()
        #accuracies.append(accuracy)
        if j%100==0:
            print("batch", str(j), "accuracy", str(accuracy) )
import numpy as np
print("accuracy of entire set", str(np.mean(accuracies)))
Execution output
0KB
	Stream
		batch 100 accuracy 0.94
		batch 200 accuracy 0.96
		batch 300 accuracy 0.94
		batch 400 accuracy 0.98
		batch 500 accuracy 0.92
		batch 600 accuracy 0.9
		batch 700 accuracy 0.96
		batch 800 accuracy 0.96
		batch 900 accuracy 0.88
		batch 1000 accuracy 0.9
		batch 1100 accuracy 0.88
		batch 1200 accuracy 0.96
		accuracy of entire set 0.93768334

Text cell <okoqZBrXgXWy>
# %% [markdown]
<h3>Visualization</h3>


Text cell <np-Qtv0DgXWy>
# %% [markdown]
Do you want to look at all the filters?


Code cell <tTZpawNUgXWy>
# %% [code]
kernels = tf.reshape(tf.transpose(W_conv1, perm=[2, 3, 0,1]),[32, -1])

Code cell <xpnYc-6mgXWz>
# %% [code]
!wget --output-document utils1.py https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0120EN-SkillsNetwork/labs/Week2/data/utils.py
import utils1
import imp
imp.reload(utils1)
from utils1 import tile_raster_images
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
image = Image.fromarray(tile_raster_images(kernels.numpy(), img_shape=(5, 5) ,tile_shape=(4, 8), tile_spacing=(1, 1)))
### Plot image
plt.rcParams['figure.figsize'] = (18.0, 18.0)
imgplot = plt.imshow(image)
imgplot.set_cmap('gray')
Execution output
1KB
	Error
		ModuleNotFoundError
		---------------------------------------------------------------------------
		ModuleNotFoundError                       Traceback (most recent call last)
		Cell In[52], line 2
		      1 get_ipython().system('wget --output-document utils1.py https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0120EN-SkillsNetwork/labs/Week2/data/utils.py')
		----> 2 import utils1
		      3 import imp
		      4 imp.reload(utils1)

		ModuleNotFoundError: No module named 'utils1'
	Stream
		'wget' is not recognized as an internal or external command,
		operable program or batch file.

Text cell <6QNkDp_dgXWz>
# %% [markdown]
Do you want to see the output of an image passing through first convolution layer?


Code cell <eeo-Bs9WgXWz>
# %% [code]
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (5.0, 5.0)
sampleimage = [x_image_train[0]]
plt.imshow(np.reshape(sampleimage,[28,28]), cmap="gray")


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Build the CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])


# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#model summary
print(model.summary)

# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.2, verbose=2)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.2f}")

# Save the model
model.save('mnist_cnn_model.h5')


In [ ]:
model.evaluate(x_test, y_test)